In [5]:
# import libraries
import pandas as pd
import random
from sklearn.feature_extraction.text import CountVectorizer
# read csv file, generate list and messages as seperate labels
df = pd.read_csv ('spam.csv', encoding = "ISO-8859-1", usecols=[0, 1])
labels = df['v1'].tolist()
label_transform = {"ham": 0, 'spam': 1}
labels = [label_transform[label] for label in labels]
messages = df['v2'].tolist()

# bag of words representation
bow_vectorizer = CountVectorizer(stop_words='english')
bow_messages = bow_vectorizer.fit_transform(messages).todense()
bow_messages = [b.tolist()[0] for b in bow_messages]

# seperate dataset into training set and validation set
all_indexs = [i for i in range(len(labels))]

# choose 75% of messages 
training_indexs = random.sample(all_indexs, k=int(len(labels)*0.75))
validation_indexs = list(set(all_indexs) - set(training_indexs))
training_labels = [labels[i] for i in training_indexs]
training_messages = [bow_messages[i] for i in training_indexs]
validation_labels = [labels[i] for i in validation_indexs]
validation_messages = [bow_messages[i] for i in validation_indexs]

print('number of messages: ', len(bow_messages))
print('number of training mesaages: ', len(training_messages))
print('number of validation messages: ', len(validation_messages))

number of messages:  5572
number of training mesaages:  4179
number of validation messages:  1393


In [6]:
import random
import math

def sub_sampling(dataset, sample_len):
    sample = []
    while len(sample) < sample_len:
        index = random.randrange(len(dataset))
        sample.append(dataset[index])
    return sample

def gini_impurity(nodes, all_labels):
    # calculate the weighted sum of gini impurity for all the nodes
    total_number_of_samples = float(sum([len(node) for node in nodes]))
    gini_impurity = 0.0
    for node in nodes:
        node_size = len(node)
        if node_size == 0:
            continue
        score = 1.0
        node_labels = [n[-1] for n in node]
        for label in all_labels:
            possibility = node_labels.count(label) / node_size
            score -= possibility ** 2
        gini_impurity += score * (node_size / total_number_of_samples)
    return gini_impurity

def train_tree_helper(node, min_size, max_depth, features_to_split, current_depth):
    left, right = node['nodes']
    del(node['nodes'])
    if not left or not right:
        # no longer splittable
        node['left'] = node['right'] = dominating_label(left + right)
        return
    if current_depth > max_depth:
        node['left'] = dominating_label(left)
        node['right'] = dominating_label(right)
        return
    if len(left) <= min_size:
        node['left'] = dominating_label(left)
    else:
        node['left'] = choose_split(left, features_to_split)
        train_tree_helper(node['left'], min_size, max_depth, features_to_split, current_depth+1)
    if len(right) <= min_size:
        node['right'] = dominating_label(right)
    else:
        node['right'] =  choose_split(right, features_to_split)
        train_tree_helper(node['right'], min_size, max_depth, features_to_split, current_depth+1)

def train_tree(sample_set, min_size, max_depth, num_features):
    # train individual trees in the forst
    print("training")
    all_features = [i for i in range(len(sample_set[0])-1)]
    features_to_split = random.sample(all_features, num_features)
    tree_root = choose_split(sample_set, features_to_split)
    train_tree_helper(tree_root, min_size, max_depth, features_to_split, 1)
    return tree_root

def split_data(sample_set, feature_ind, val):
    left = [sample for sample in sample_set if sample[feature_ind] < val]
    right = [sample for sample in sample_set if sample[feature_ind] >= val]
    return (left, right)

def choose_split(sample_set, features_to_split):
    all_labels = list(set([sample[-1] for sample in sample_set]))
    final_splitting_index = None
    final_splitting_val = None
    final_splitting_nodes = None
    best_score = math.inf
    for feature in features_to_split:
        splitting_vals = set([s[feature] for s in sample_set])
        for splitting_val in splitting_vals:
            nodes = split_data(sample_set, feature, splitting_val)
            gini_score = gini_impurity(nodes, all_labels)
            if gini_score < best_score:
                final_splitting_index = feature
                final_splitting_val = splitting_val
                final_splitting_nodes = nodes
                best_score = gini_score
    return {'index': final_splitting_index, 'val': final_splitting_val, 'nodes':final_splitting_nodes}

def dominating_label(sample_set):
    labels = [sample[-1] for sample in sample_set]
    return max(set(labels), key = labels.count)
    

def treePrediction(sample, tree):
    if sample[tree['index']] < tree['val']:
        # left
        if isinstance(tree['left'], dict):
            return treePrediction(sample, tree['left'])
        else:
            return tree['left']
    else:
        if isinstance(tree['right'], dict):
            return treePrediction(sample, tree['right'])
        else:
            return tree['right']
        
def multipleTreesPrediction(sample, trees):
    predictions = []
    for tree in trees:
        predictions.append(treePrediction(sample, tree))
    return max(set(predictions), key = predictions.count)

def randomForest(train_set,train_label, min_size, max_depth, num_trees, num_features, sample_size):
    # train_set -> the training data
    # min_size -> the minimum number of samples in a non-terminal node
    # max_depth -> the maximize number of layers of the tree
    # num_trees -> number of sub-trees to train
    # num_features -> number of features to split on 
    # sample_size -> the size of the sub-sample we create for each tree
    padded_train_set = [train_set[i] + [train_label[i]] for i in range(len(train_set))]
    trees = []
    for i in range(num_trees):
        print('generating tree: ', i)
        sub_sample = sub_sampling(padded_train_set, sample_size)
        new_tree = train_tree(sub_sample, min_size, max_depth, num_features)
        trees.append(new_tree)
    return trees
                
        

In [7]:
trees = randomForest(training_messages, training_labels, 2, 50, 10, 2000, 1000)

generating tree:  0
training


KeyboardInterrupt: 

In [18]:
re = []
for i, sample in enumerate(validation_messages):
    print("processing message: ", i)
    re.append(multipleTreesPrediction(sample, trees))
    
print(len(re))

processing message:  0
processing message:  1
processing message:  2
processing message:  3
processing message:  4
processing message:  5
processing message:  6
processing message:  7
processing message:  8
processing message:  9
processing message:  10
processing message:  11
processing message:  12
processing message:  13
processing message:  14
processing message:  15
processing message:  16
processing message:  17
processing message:  18
processing message:  19
processing message:  20
processing message:  21
processing message:  22
processing message:  23
processing message:  24
processing message:  25
processing message:  26
processing message:  27
processing message:  28
processing message:  29
processing message:  30
processing message:  31
processing message:  32
processing message:  33
processing message:  34
processing message:  35
processing message:  36
processing message:  37
processing message:  38
processing message:  39
processing message:  40
processing message:  41
pr

processing message:  339
processing message:  340
processing message:  341
processing message:  342
processing message:  343
processing message:  344
processing message:  345
processing message:  346
processing message:  347
processing message:  348
processing message:  349
processing message:  350
processing message:  351
processing message:  352
processing message:  353
processing message:  354
processing message:  355
processing message:  356
processing message:  357
processing message:  358
processing message:  359
processing message:  360
processing message:  361
processing message:  362
processing message:  363
processing message:  364
processing message:  365
processing message:  366
processing message:  367
processing message:  368
processing message:  369
processing message:  370
processing message:  371
processing message:  372
processing message:  373
processing message:  374
processing message:  375
processing message:  376
processing message:  377
processing message:  378


processing message:  690
processing message:  691
processing message:  692
processing message:  693
processing message:  694
processing message:  695
processing message:  696
processing message:  697
processing message:  698
processing message:  699
processing message:  700
processing message:  701
processing message:  702
processing message:  703
processing message:  704
processing message:  705
processing message:  706
processing message:  707
processing message:  708
processing message:  709
processing message:  710
processing message:  711
processing message:  712
processing message:  713
processing message:  714
processing message:  715
processing message:  716
processing message:  717
processing message:  718
processing message:  719
processing message:  720
processing message:  721
processing message:  722
processing message:  723
processing message:  724
processing message:  725
processing message:  726
processing message:  727
processing message:  728
processing message:  729


processing message:  1093
processing message:  1094
processing message:  1095
processing message:  1096
processing message:  1097
processing message:  1098
processing message:  1099
processing message:  1100
processing message:  1101
processing message:  1102
processing message:  1103
processing message:  1104
processing message:  1105
processing message:  1106
processing message:  1107
processing message:  1108
processing message:  1109
processing message:  1110
processing message:  1111
processing message:  1112
processing message:  1113
processing message:  1114
processing message:  1115
processing message:  1116
processing message:  1117
processing message:  1118
processing message:  1119
processing message:  1120
processing message:  1121
processing message:  1122
processing message:  1123
processing message:  1124
processing message:  1125
processing message:  1126
processing message:  1127
processing message:  1128
processing message:  1129
processing message:  1130
processing m

In [20]:
false = []
for i in range(len(re)):
    if re[i] != validation_labels[i]:
        false.append((i, re[i], validation_labels[i]))
print("number of false predictions: ", len(false))
print(false)

number of false predictions:  79
[(22, 0, 1), (40, 0, 1), (63, 0, 1), (83, 0, 1), (100, 0, 1), (107, 0, 1), (140, 0, 1), (143, 0, 1), (148, 0, 1), (172, 0, 1), (189, 0, 1), (192, 0, 1), (205, 0, 1), (208, 0, 1), (217, 0, 1), (278, 0, 1), (302, 0, 1), (324, 0, 1), (369, 0, 1), (387, 0, 1), (391, 0, 1), (426, 0, 1), (462, 0, 1), (500, 0, 1), (507, 0, 1), (533, 0, 1), (538, 0, 1), (541, 0, 1), (549, 0, 1), (559, 0, 1), (560, 0, 1), (589, 0, 1), (591, 0, 1), (596, 0, 1), (611, 0, 1), (624, 0, 1), (699, 0, 1), (706, 0, 1), (708, 0, 1), (718, 0, 1), (753, 0, 1), (793, 0, 1), (835, 0, 1), (840, 0, 1), (868, 0, 1), (886, 0, 1), (896, 0, 1), (912, 0, 1), (927, 0, 1), (931, 0, 1), (945, 0, 1), (983, 0, 1), (986, 0, 1), (993, 0, 1), (1007, 0, 1), (1013, 0, 1), (1028, 0, 1), (1029, 0, 1), (1031, 0, 1), (1077, 0, 1), (1115, 0, 1), (1116, 0, 1), (1122, 0, 1), (1123, 0, 1), (1141, 0, 1), (1166, 0, 1), (1175, 0, 1), (1183, 0, 1), (1248, 0, 1), (1261, 0, 1), (1266, 0, 1), (1282, 0, 1), (1284, 0, 1), (1